In [1]:
# Structurally Constrained Recurrent Network (SCRN) Model
#
# This gives an implementation of the SCRN model given in Mikolov et al. 2015, arXiv:1412.7753 [cs.NE], 
# https://arxiv.org/abs/1412.7753 using Python and Tensorflow.
#
# This IPython Notebook provides an example of how to call the associated library of Python scripts.  
# Mikolov et al. should be consulted to make sure of the correct hyperparameter values.
#
# Stuart Hagler, 2017

In [2]:
# Imports
import math
import sys

# Local Imports
sys.path.insert(0, 'py')
from lstm import lstm_graph
from read_data import read_data
from scrn import scrn_graph
from srn import srn_graph
from tokens import text_elements_to_tokens

In [3]:
# Flags
rnn_flg = 3      # 1 for SRN
                 # 2 for LSTM
                 # 3 for SCRN
usecase_flg = 1  # 1 for predicting letters
                 # 2 for predicting words with cutoff for infrequent words

In [4]:
# Network-specific hyperparameters
if rnn_flg == 1:
    
    # Network hyperparameters
    hidden_size = 110               # Dimension of the hidden vector
    
    # Training hyperparameters
    num_training_unfoldings = 10    # Number of training unfoldings
    state_size = []                 # Dimension of the state vector
    
elif rnn_flg == 2:
    
    # Network hyperparameters
    hidden_size = 110               # Dimension of the hidden vector
    state_size = hidden_size        # Dimension of the state vector
    
    # Training hyperparameters
    num_training_unfoldings = 10    # Number of training unfoldings
    
elif rnn_flg == 3:
    
    # Network hyperparameters
    alpha = 0.95                    #
    hidden_size = 100               # Dimension of the hidden vector
    state_size = 10                 # Dimension of the state vector

    # Training hyperparameters
    num_training_unfoldings = 50    # Number of training unfoldings
    
# General network hyperparameters
word_frequency_cutoff = 50          # Cutoff for infrequent words for usecase_flg = 2

# General training hyperparameters
base_training_batch_size = 32       # Training batch size across all towers
clip_norm = 1.25                    # Norm for gradient clipping
learning_decay = 1/1.5              # Multiplier to decay the learn rate when required
learning_rate = 0.05                # Initial learning rate
momentum = 0.9                      # Momentum for training
num_epochs = 50                     # Total number of epochs to run the algorithm
num_validation_unfoldings = 50      # Number of validation unfoldings
optimization_frequency = 5          # Number of unfoldings before optimization step
summary_frequency = 500             # Summary information is displayed after training this many batches
validation_batch_size = 32          # Validation batch size for each tower

# Cluster
num_gpus = 1                        # Number of GPUs available

# Logging
logdir = '/tmp/tensorflow/log/'

# Data file
filename = 'data/text8.zip'

In [5]:
# Prepare training, validation, test data sets
num_towers = num_gpus
training_batch_size = base_training_batch_size // num_towers
raw_data = read_data(usecase_flg, filename)
data, dictionary, reverse_dictionary, vocabulary_size = text_elements_to_tokens(usecase_flg, raw_data, 
                                                                                word_frequency_cutoff)
training_size = math.floor((9/11)*len(raw_data)/num_towers)
validation_size = math.floor((1/11)*len(raw_data)/num_towers)
testing_size = math.floor((1/11)*len(raw_data)/num_towers)
training_text = []
validation_text = []
testing_text = []
for i in range(num_towers):
    training_text.append(data[i*training_size:(i+1)*training_size])
    validation_text.append(data[num_towers*training_size + i*validation_size: \
                                num_towers*training_size + (i+1)*validation_size])
    testing_text.append(data[num_towers*(training_size + validation_size) + i*testing_size: \
                             num_towers*(training_size + validation_size) + (i+1)*testing_size])

In [6]:
print('Vocabulary Size: %d' % vocabulary_size)

# Initialize graph
if rnn_flg == 1:
    # Use SRN
    graph = srn_graph(num_gpus, hidden_size, state_size, vocabulary_size, num_training_unfoldings, 
                      num_validation_unfoldings, training_batch_size, validation_batch_size, optimization_frequency)
elif rnn_flg == 2:
    # Use LSTM
    graph = lstm_graph(num_gpus, hidden_size, state_size, vocabulary_size, num_training_unfoldings, 
                       num_validation_unfoldings, training_batch_size, validation_batch_size, optimization_frequency)
elif rnn_flg == 3:
    # Use SCRN
    graph = scrn_graph(num_gpus, alpha, hidden_size, state_size, vocabulary_size, num_training_unfoldings, 
                       num_validation_unfoldings, training_batch_size, validation_batch_size, optimization_frequency)
    
# Train graph
graph.train(learning_rate, learning_decay, momentum, clip_norm, num_epochs, summary_frequency, training_text, 
            validation_text, testing_text, logdir)

Vocabulary Size: 28
Initialized


KeyboardInterrupt: 